In [ ]:
# import python packages
import scipy
from scipy import optimize
import os
import numpy as np
import os.path
import time
import matplotlib.pyplot as plt

In [ ]:
# define function called fun() that is called by scipy.optimize.least_squares()
def fun(x):
    global count
    Ploading, Ea = x
    Ploading = Ploading / 1000
    new_parameters = 'Ea = ' + str(Ea) + '\n', 'Ploading = ' + str(Ploading) + '\n'
    with open("parameters.txt", "w") as file:
        file.writelines(new_parameters)
    with open("parameters_archive.txt", "a") as file:
        file.writelines(str(count)+'\t'+str(Ea)+'\t'+str(Ploading)+'\n')
    
    width = []
    dP = 2
    while dP in range(2,13):
        dP1 = dP
        dP2 = dP+1
        
        # set wd
        os.system('L:;cd EL_final_final/optimization_24')
        
        if count <= 4:
            pass
        else:
            # write input files
            data = data2 = data3 = ""
            with open('header.txt') as fp:
                data = fp.read()
            with open('parameters.txt') as fp:
                data2 = fp.read()
            with open('body_'+str(dP1)+'.txt') as fp:
                data3 = fp.read()
            data = data + data2 + data3
            filename1 = 'extrusionLoadingPS_'+str(dP1)+'_'+str(count)+'.inp'
            with open (filename1, 'w') as fp:
                fp.write(data)
            os.system('abaqus job=' + filename1)
        
            time.sleep(5)
        
            if dP2 <= 12:
                data = data2 = data3 = ""  
                with open('header.txt') as fp:
                    data = fp.read()
                with open('parameters.txt') as fp:
                    data2 = fp.read()
                with open('body_'+str(dP2)+'.txt') as fp:
                    data3 = fp.read()
                data = data + data2 + data3
                filename2 = 'extrusionLoadingPS_'+str(dP2)+'_'+str(count)+'.inp'
                with open (filename2, 'w') as fp:
                    fp.write(data)
                os.system('abaqus job=' + filename2)
    
        file_path1 = 'L:/EL_final_final/optimization_24/extrusionLoadingPS_%s.lck'%(str(dP1)+'_'+str(count))
        file_path2 = 'L:/EL_final_final/optimization_24/extrusionLoadingPS_%s.lck'%(str(dP2)+'_'+str(count))

        while os.path.exists(file_path1) or os.path.exists(file_path2):
            time.sleep(180)
        
        # modify macro file
        pathName1 = "pathName = 'L:/EL_final_final/optimization_24/extrusionLoadingPS_%s.odb'"%(str(dP1)+'_'+str(count))
        with open("macro_path.txt", "w") as file:
            file.writelines(pathName1)    
        data = data2 = data3 = ""  
        with open('macro_header.txt') as fp:
            data = fp.read()
        with open('macro_path.txt') as fp:
            data2 = fp.read()
        with open('macro_body.txt') as fp:
            data3 = fp.read()
        data = data + '\n' + data2 + '\n' + data3
        filename1 = 'getWidth1.py'
        with open (filename1, 'w') as fp:
            fp.write(data)

        # output report file from abaqus
        os.system('abaqus cae noGUI=getWidth1.py')
        
        time.sleep(5)

        # read report file and load deformed width
        with open('abaqus.rpt') as file:
            data = file.readlines()[-5]
        width = width + [float(data.split()[1]) * 2]
    
        time.sleep(5)
    
        if dP2<=12:
            # modify macro file
            pathName2 = "pathName = 'L:/EL_final_final/optimization_24/extrusionLoadingPS_%s.odb'"%(str(dP2)+'_'+str(count))
            with open("macro_path.txt", "w") as file:
                file.writelines(pathName2)    
            data = data2 = data3 = ""  
            with open('macro_header.txt') as fp:
                data = fp.read()
            with open('macro_path.txt') as fp:
                data2 = fp.read()
            with open('macro_body.txt') as fp:
                data3 = fp.read()
            data = data + '\n' + data2 + '\n' + data3
            filename2 = 'getWidth2.py'
            with open (filename2, 'w') as fp:
                fp.write(data)

            # output report file from abaqus
            os.system('abaqus cae noGUI=getWidth2.py')
                
            time.sleep(5)

            # read report file and load deformed width
            with open('abaqus.rpt') as file:
                data = file.readlines()[-5]
            width = width + [float(data.split()[1]) * 2]
            
            time.sleep(5)
        
        dP = dP+2
    
    # residual
    true_width = [0.8831,0.8690,0.8670,0.7724,0.8002,0.7394,0.6468,0.5993,0.5842,0.5411,0.5351]
    diff = [a - b for a, b in zip(true_width,width)]
    residuals = np.array(diff)
    width_report = np.array(width)
    
    with open("residuals_archive.txt", "a") as file:
        if count == 1:
            file.writelines(str(count)+'\t')
        else:
            file.writelines('\n'+str(count)+'\t')
        for res in residuals:
            file.writelines(str(res)+'\t')
    
    with open("width_archive.txt", "a") as file:
        if count == 1:
            file.writelines(str(count)+'\t')
        else:
            file.writelines('\n'+str(count)+'\t')
        for w in width_report:
            file.writelines(str(w)+'\t')
            
    # count up simulation
    count += 1
    
    return residuals

In [ ]:
# initiate text archive files containing parameter values and mse values
with open('parameters_archive.txt','w') as file:
    file.writelines('count'+'\t'+'Ea'+'\t'+'Ploading'+'\n')
with open('residuals_archive.txt','w') as file:
    file.writelines('count'+'\t'+'residuals'+'\n')
with open('width_archive.txt','w') as file:
    file.writelines('count'+'\t'+'widths'+'\n')

# initialize count variable to count up number of parameter sets
count = 1

# call optimization function
res = scipy.optimize.least_squares(fun, x0=(150,4), bounds=([0,0],[1000,20]), gtol=1e-02, diff_step=0.05, verbose=2)

In [ ]:
# check optimization result
res